This is the start of the Notebook.
We are going to start by defining the skeleton given to us on Moodle for this assignment. It starts with the following imports:


In [1]:
import gym
import random
import numpy as np
from random import randint
from statistics import mean, median
import math

In [1048]:
env = gym.make("CartPole-v0")
env.reset()

array([ 0.00461606,  0.04271656, -0.03567026,  0.02977947])

This answers the 1st question of the assignment, which comprises of initializing the parameters needed to run the whole algorithm:

In [1069]:
goal_steps = 500
mutationRate = 20
crossoverRate = 85
score_requirement = 20
data_points = 20000
num_solutions = 10
num_generations = 40
test_size = 20

We will now proceed to initialize the population that we will start with:

In [1060]:
def create_initial_pop(pop_size):
    return np.random.uniform(low = -2.0, high = 2.0, size = pop_size)

Next up, we will work on implementing the functions we discussed during class: 

In [1061]:
def call_fitness(population, X, y, pop_size):
    a = pop_size[0]
    b = 1
    fit = np.empty((a, b))
    for i in range(a):
        fit[i][0] = np.sum(np.dot(X, (population[i]).T))
    return fit

In [1062]:
def selection(population, fitness, num_parents):
    fit = list(fitness)
    init_fit = fit[0]
    parents_placeholder = np.empty((num_parents, population.shape[1]))
    for i in range(num_parents):
        max_i = np.where(fit == np.max(fit))
        parents_placeholder[i,:] = population[max_i[0][0], :]
        fit[max_i[0][0]] = -math.inf
    return parents_placeholder

In [1063]:
def crossover(parents, num_offsprings):
    returnV = np.empty((num_offsprings, parents.shape[1]))
    pt1 = int(parents.shape[1]/2)
    i=0
    while (parents.shape[0] < num_offsprings):
        i1 = i%parents.shape[0]
        i2 = (i+1)%parents.shape[0]
        x = random.random()
        if x > (crossoverRate//100):
            continue
        i1 = i%parents.shape[0]
        i2 = (i+1)%parents.shape[0]
        returnV[i,0:pt1] = parents[i1,0:pt1]
        returnV[i,pt1:] = parents[i2,pt1:]
        i=+1
    return returnV 

In [1064]:
def mutation(offsprings):
    vals = np.empty((offsprings.shape))
    n_vals = vals.shape[0]
    for i in range(n_vals):
        x = random.random()
        vals[i,:] = offsprings[i,:]
        if x > mutationRate/100:
            continue
        x = randint(0,offsprings.shape[1]-1)    
        vals[i,x] += np.random.uniform(-1.0, 1.0, 1)  
    return vals

In the text cell, we will implemented the actual GA model:

In [1070]:
def GA_model():
    scores = []
    training_data = []
    for i in range(data_points):
        reward_i = 0
        history = []
        old_obs_hist = []
        for i in range(goal_steps):
            bit = random.randrange(0,2)
            observation, reward, done, info = env.step(bit)
            
            reward_i += reward
            
            if old_obs_hist != []:
                history.append([old_obs_hist, bit])

            old_obs_hist = observation
            
            if done:
                break
            
        if score_requirement < reward_i:
            for data in history:
                training_data.append(data)
            scores.append(reward_i)
                    
        env.reset()

    X = np.array([i[0] for i in training_data])
    y = np.array([i[1] for i in training_data]).reshape(-1, 1)

    weights = []
    pop_size = (num_solutions, X.shape[1])
    num_parents = int(pop_size[0]/2)
    num_offsprings = int(pop_size[0]/2)

    population = create_initial_pop(pop_size)

    for i in range(num_generations):
        fitness = call_fitness(population, X, y, pop_size)
        parents = selection(population, fitness, num_parents)
        offsprings = crossover(parents, num_offsprings)
        mutants = mutation(offsprings)
        population[0:parents.shape[0], :] = parents
        population[parents.shape[0]:, :] = mutants

    max_fitness = np.where(call_fitness(population, X, y, pop_size) == np.max(call_fitness(population, X, y, pop_size)))
    weights.append(population[max_fitness[0][0],:])
    return weights

weights = np.asarray(GA_model())

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarr

The next cell works testing the GA model implemented above:

In [1071]:
def test_helper(X):
    a = X.shape[0]
    b = 1
    pred = np.empty((a, b))
    for i in range(a):
        if X[i] < 0.5:
            val = 1
        else:
            val = 0
        pred[i]=val
    return pred  

def test(data, w):
    val = 1/(1+np.exp(-(data@(w).T)))
    ret = test_helper(val)
    ret = ret.astype(int)
    return ret[0][0]

all_scores=[]

for _ in range(test_size):
    current_score = 0
    old = []
    env.reset()
    for _ in range(goal_steps):
        # env.render()
        if old == []:
            action = random.randrange(0,2)
        else:
            action = test(old, weights) 
        new_observation, reward, done, info = env.step(action)
        old = new_observation
        current_score += reward
        if done:
            break
    all_scores.append(current_score)        
print('Average Score Achieved: {}, Initial Score Achieved: {}, Final Score Achieved: {}.'.format(mean(all_scores), all_scores[0], all_scores[-1]))
if mean(all_scores)>score_requirement:
  print("GA Algorithm is succesfull!")
else:
  print("Run Again!")
env.close()

Average Score Achieved: 200.0, Initial Score Achieved: 200.0, Final Score Achieved: 200.0.
GA Algorithm is succesfull!


We will now change some of the initial values, and see how it might affect our results:

In [1082]:
goal_steps = 250
mutationRate = 40
crossoverRate = 85
score_requirement = 40
data_points = 10000
num_solutions = 10
num_generations = 25
test_size = 25

In [1083]:
weights = np.asarray(GA_model())

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarr

In [1084]:
all_scores=[]

for _ in range(test_size):
    current_score = 0
    old = []
    env.reset()
    for _ in range(goal_steps):
        if old == []:
            action = random.randrange(0,2)
        else:
            action = test(old, weights) 
        new_observation, reward, done, info = env.step(action)
        old = new_observation
        current_score += reward
        if done:
            break
    all_scores.append(current_score)        
print('Average Score Achieved: {}, Initial Score Achieved: {}, Final Score Achieved: {}.'.format(mean(all_scores), all_scores[0], all_scores[-1]))
if mean(all_scores)>score_requirement:
  print("GA Algorithm is succesfull!")
else:
  print("Run Again!")
env.close()

Average Score Achieved: 103.04, Initial Score Achieved: 179.0, Final Score Achieved: 200.0.
GA Algorithm is succesfull!


We can see that the scores where affected negatively, but our run was still successful even when we made the scoring requirement higher than the first run. It is difficult to determine what parameter really affected the performance of our algorithm, since we changed many at once without having a benchmark. In general, this could be caused by the smaller number of data points from which the algorithm would learn, and a smaller number of generation could also be causing this, as more computations could have potentially meant a better performance. The crossover and mutation rate could also play a role.


We will now try a new fitness function and implement it. We will use the pairwise distances between the 2 arrays to get the actual fitness function. We will then get:

In [1102]:
# import sklearn.metrics.pairwise_distances
from sklearn.metrics import pairwise_distances

def call_fitness(population, X, y, pop_size):
    a = pop_size[0]
    b = 1
    fit = np.empty((a, b))
    X = X.reshape(-1,1)
    for i in range(a):
        y =(population[i]).T.reshape(-1,1)
        fit[i][0] = np.sum(pairwise_distances(X, y))
    return fit

In [ ]:
weights = np.asarray(GA_model())
print(weights)

In [1105]:
weights = np.asarray(GA_model())

all_scores=[]

for _ in range(test_size):
    current_score = 0
    old = []
    env.reset()
    for _ in range(goal_steps):
        if old == []:
            action = random.randrange(0,2)
        else:
            action = test(old, weights) 
        new_observation, reward, done, info = env.step(action)
        old = new_observation
        current_score += reward
        if done:
            break
    all_scores.append(current_score)        
print('Average Score Achieved: {}, Initial Score Achieved: {}, Final Score Achieved: {}.'.format(mean(all_scores), all_scores[0], all_scores[-1]))
if mean(all_scores)>score_requirement:
  print("GA Algorithm is succesfull!")
else:
  print("Run Again!")
env.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray 

Average Score Achieved: 129.36, Initial Score Achieved: 200.0, Final Score Achieved: 64.0.
GA Algorithm is succesfull!


As we can see it is performing well enough, with the average score achieving 130, and the first one achieving the max score of 200.

We will try another similarity function, which is the cosine similarity function:

In [1108]:
# import sklearn.metrics.pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

def call_fitness(population, X, y, pop_size):
    a = pop_size[0]
    b = 1
    fit = np.empty((a, b))
    X = X.reshape(-1,1)
    for i in range(a):
        y =(population[i]).T.reshape(-1,1)
        fit[i][0] = np.sum(cosine_similarity(X,y))
    return fit

In [1114]:
weights = np.asarray(GA_model())
print(weights)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarr

[[-0.62939961 -0.86420264 -0.61065309 -0.86274893]]


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [1115]:
all_scores=[]

for _ in range(test_size):
    current_score = 0
    old = []
    env.reset()
    for _ in range(goal_steps):
        if old == []:
            action = random.randrange(0,2)
        else:
            action = test(old, weights) 
        new_observation, reward, done, info = env.step(action)
        old = new_observation
        current_score += reward
        if done:
            break
    all_scores.append(current_score)        
print('Average Score Achieved: {}, Initial Score Achieved: {}, Final Score Achieved: {}.'.format(mean(all_scores), all_scores[0], all_scores[-1]))
if mean(all_scores)>score_requirement:
  print("GA Algorithm is succesfull!")
else:
  print("Run Again!")
env.close()

Average Score Achieved: 135.32, Initial Score Achieved: 77.0, Final Score Achieved: 93.0.
GA Algorithm is succesfull!


This also achieves a successful run. 

We will now handle the last question of the assignment, regarding the difference between an RL solution and a GA solution.

We will start by a quick reminder of the definitions of RL and GA. RL is a training process for Machine Learning models to eventually take decisions , by utilizing an agent, in an environment, using a reward system. GA is a search "metaheuristic" that is based on a selection process that selects the fittest data samples and reporduces based on these samples. We can see from the defintions the difference is obvious, where in RL there is non-stop learning from the agent whereas GA some agents have to die in order for the others to learn. In addition, RL is more specific for problems in which we can find some policy and a reward system related to the policy. However, GA can be applied to any optimization problem as we can find a fitness function to characterize the problem and change the solution accordingly. Each also has is drawbacks, where we know that in GA, finding and coding of the fitness function and other functions related to GA can be challenging and it is expensive when it comes to the amount of computation done, and for RL, we need a lot of data, it is less efficient for real life problems and simple problems and is computationally heavy. This means that RL is suitable for some problems, while GA is more suitable for others. Also, there has been some instances of both being used simultaneously.
In the proposed solution, we have defined the GA solution using the selection, the creation of new population, the fitness function and the remaining functions. An RL solution would conceptually include the definition of he environment, the possible actions (going left or right), the states that we could get to that will be mapped to actions learned by the algorithm based on a reward system we also define. The reward system could be +0 for all actions, except the fall -10 for example, or +5 for every successful move and -10 for the fall, or any system that would make sense and that will get use to our wanted solution.

Resources: 
- https://kth.diva-portal.org/smash/get/diva2:1358693/FULLTEXT01.pdf    
- https://www.researchgate.net/publication/268010749_Genetic_Algorithm_-_Survey_Paper
- https://gym.openai.com/docs/
- https://medium.com/koderunners/genetic-algorithm-part-1-intuition-fde1b75bd3f9 (thread)

